# Notebook for Work with Optimization and Pool and Executor instances
For possible application in ModelFitting in brian2tools

Notebook goals: 
    - create instances with Scipy and Nevergrad that take two params and return optimal error
    - modify them to use the workers with Executor and Pool 
    - try using NeuronGrouop to adapt them

In [1]:
import numpy as np
import multiprocessing as mp

from scipy.optimize import differential_evolution, rosen
from scipy.optimize._differentialevolution import DifferentialEvolutionSolver

from nevergrad.optimization import optimizerlib
from concurrent import futures
from nevergrad import instrumentation as inst

## Scipy

In [2]:
def test(x):
    arg2 = x[0] - x[1]
    return arg2

In [3]:
bounds = [(-5, 5), (-5, 5)]
result = differential_evolution(test, bounds)
result.x, result.fun

(array([-5.,  5.]), -10.0)

### use workers to map

In [4]:
with mp.Pool(2) as p, DifferentialEvolutionSolver(
    test, bounds, updating='deferred', workers=p.map) as solver:
    solver.solve()

In [5]:
print(solver.x)
print(test(solver.x))

[-5.  5.]
-10.0


In [6]:
with DifferentialEvolutionSolver(
    test, bounds, updating='deferred', workers=mp.Pool(2).map) as solver:
    solver.solve()

In [7]:
differential_evolution(test, bounds, updating='deferred', workers=mp.Pool(2).map)

     fun: -10.0
     jac: array([ 1.00000008, -1.00000008])
 message: 'Optimization terminated successfully.'
    nfev: 726
     nit: 23
 success: True
       x: array([-5.,  5.])

In [8]:
print(solver.x)
print(test(solver.x))

[-5.  5.]
-10.0


### Using The Nevergrad PoolExecutor with Scipy optimize

In [10]:
with futures.ProcessPoolExecutor(max_workers=2) as p, DifferentialEvolutionSolver(
    test, bounds, updating='deferred', workers=p.map) as solver:
    solver.solve()

In [11]:
print(solver.x)
print(test(solver.x))

[-5.  5.]
-10.0


### Nevergrad

In [12]:
# define the variables to be a bounded array
arg1 = inst.var.Array(2).bounded(-5, 5)

instrum = inst.Instrumentation(arg1)
instrum

Instrumentation(Array(shape=(2,), transforms=[ArctanBound(a_max=5, a_min=-5)]))

In [13]:
# pick the optimization method; budget = number of allowed evaluations
optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=5000)

In [14]:
optim.optimize(test)

Candidate(args=(array([-4.99985122,  4.99924018]),), kwargs={}, data=[-21394.25805858   4189.25640781])

### Nevergrad with Executor

In [15]:
optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=10000, num_workers=2)

with futures.ProcessPoolExecutor(max_workers=optim.num_workers) as executor:
    recommendation = optim.optimize(test, executor=executor, batch_mode=True)

/home/alteska/anaconda3/envs/brian_opt/lib/python3.6/site-packages/nevergrad/common/tools.py:168: UserWarning: Ignoring since timer was stopped before starting.
  warnings.warn("Ignoring since timer was stopped before starting.")


In [16]:
recommendation

Candidate(args=(array([-5.,  5.]),), kwargs={}, data=[-1.47296485e+11  9.19212462e+11])

In [17]:
# can not be used the other way around, needs "submit" method

# optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=10000, num_workers=2)
# with mp.Pool(processes=optim.num_workers) as executor:
#     recommendation = optim.optimize(test, executor=executor, batch_mode=True)

### Two Separate Bounds Problem

In [18]:
arg1 = inst.var.Array(1).bounded(-5,5)
arg2 = inst.var.Array(1).bounded(0,10)
instrum = inst.Instrumentation(arg1, arg2)


In [19]:
def test2(x, y):
    arg2 = x - y
    return arg2

In [21]:
# pick the optimization method; budget = number of allowed evaluations
optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=5000)
optim.optimize(test)

TypeError: test() takes 1 positional argument but 2 were given

### Test with More complicated function

In [ ]:
optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=10000, num_workers=2)

with futures.ProcessPoolExecutor(max_workers=optim.num_workers) as executor:
    recommendation = optim.optimize(test, executor=executor, batch_mode=True)